
# 6-year test run
# Only look at floats under ice: How much does the trajectory deviate from straight line when float is under ice?
# how much do surface properties vary along the path? 
# How many model grid cells does the trajectory cross under ice?

# look at each year individually

# Fig. 8 in GMD paper


In [ ]:

import sys
sys.path.append('/global/homes/c/cnissen/scripts/seawater-3.3.4/seawater/')
import os
import numpy as np
import xarray as xr
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from matplotlib import cm
import seawater
#from seawater import dist
#import seawater as sw
import matplotlib.path as mpath
from cartopy.util import add_cyclic_point
import matplotlib.gridspec as gridspec
import matplotlib.ticker as mticker
from cartopy.mpl.ticker import (LongitudeFormatter, LatitudeFormatter,
                                LatitudeLocator)
import random
from numba import njit
from math import sin, cos, sqrt, atan2, radians
from tqdm import tqdm
from mpasview import * # Qing's library, modified by Yohei Takano, Dec, 2022


In [ ]:
#-----
# saving plots
#-----

savepath     = '/global/cfs/cdirs/m4003/cnissen/Plots/E3SM_floats/trajectories_under_ice/6year_run/'
# check existence of paths
if not os.path.exists(savepath):
    print ('Created '+savepath)
    os.makedirs(savepath)
    
savepath2     = savepath+'example_trajectories/'
# check existence of paths
if not os.path.exists(savepath2):
    print ('Created '+savepath2)
    os.makedirs(savepath2)
    
savepath3     = savepath+'eps/'
# check existence of paths
if not os.path.exists(savepath3):
    print ('Created '+savepath3)
    os.makedirs(savepath3)
    

In [ ]:
####
# specifics for trajectory output
####

rad_to_deg = 180.0/np.pi
latlim = -45.0

path_mesh = '/global/cfs/cdirs/m4003/maltrud/'
meshID = 'EC30to60E2r2'
meshfile = xr. open_dataset(path_mesh+'ocean.'+meshID+'.210210.nc')
#print(meshfile)

lon  = meshfile['lonCell'].values*rad_to_deg
lat  = meshfile['latCell'].values*rad_to_deg
topo = meshfile['bottomDepth'].values
area = meshfile['areaCell'].values
zlevs            = meshfile['refBottomDepth'].values
layerThickness   = meshfile['layerThickness'].values
restingThickness = meshfile['restingThickness'].values

print(len(lon),'nodes in mesh')
print(topo.shape)
print(area.shape)
print('Min/Max lon:',np.min(lon),np.max(lon))
print('Min/Max lat:',np.min(lat),np.max(lat))
print('layerThickness.shape:',layerThickness.shape)
print('restingThickness.shape:',restingThickness.shape)

meshfile.close()

print(zlevs[38])


In [ ]:
#----
# load daily float output from E3SM: oxygen, nitrate
#----
# NOTE: the reduction to "deep" floats is not 100% exact.
#  -> when running this over different years, a different number of floats is identified as "shallow" floats
#  -> for now, I kick out those that are "shallow" on day 1 of year 1
# How to treat this later?

path = '/global/cfs/cdirs/m4003/maltrud/6year/floats/'
year_list = ['0055','0056','0057','0058','0059','0060']

# kick out floats in shallow regions (these are not advected I think)
ind = np.where(zlevs<=2200)[0]

for yy in range(0,len(year_list)):
    print('Load year '+year_list[yy])
    file1 = 'floats.year'+year_list[yy]+'.nc'   
    data = xr. open_dataset(path+file1)

    lon_1   = data['particleColumnLon'].values*rad_to_deg 
    lat_1   = data['particleColumnLat'].values*rad_to_deg 
    ice_1   = data['particle_ifrac'].values 
    #oxy_1   = data['particleColumnO2'].values #[:,0,:]
    no3_1   = data['particleColumnNO3'].values
    u_1   = data['particleColumnVelZonal'].values
    v_1   = data['particleColumnVelMerid'].values
    #print('lat_all',lat_all.shape)

    # set missing values to NaN (deep ocean layers) 
    #lat_1[no3_1==-1]=np.nan
    #lon_1[no3_1==-1]=np.nan
    lat_1[lat_1==0]=np.nan
    lon_1[lon_1==0]=np.nan
    no3_1[no3_1==-1]=np.nan 
    u_1[u_1==-1]=np.nan 
    v_1[v_1==-1]=np.nan 
    #oxy_1[oxy_1==-1]=np.nan 
    
    print('Reduce to floats in the deep ocean')
    if yy==0: # only load the first time, re-use ind_deep
        aux = np.sum(np.isnan(no3_1[0,ind,:]),axis=0) # check if any of the depth levels shallower than 1100m is NaN
        ind_deep = np.where(aux==0)[0] # if it is, aux is >0; only keep those that are 0
  #  #print('Floats in the deep ocean:',ind_deep.shape)
    lon_1   = lon_1[:,:,ind_deep]
    lat_1   = lat_1[:,:,ind_deep]
    no3_1   = no3_1[:,:,ind_deep]
    u_1     = u_1[:,:,ind_deep]
    v_1     = v_1[:,:,ind_deep]
    ice_1   = ice_1[:,ind_deep]
    #oxy_1   = oxy_1[:,:,ind_deep]
    
    #indSO = np.where(lat_all[0,:]<=-50)[0]
    
    if yy==0: # first time
        lat_all = np.expand_dims(lat_1[:,0,:],axis=0)
        lon_all = np.expand_dims(lon_1[:,0,:],axis=0)
        no3_all = np.expand_dims(no3_1[:,0,:],axis=0)
        u_all   = np.expand_dims(u_1[:,38,:],axis=0)
        v_all   = np.expand_dims(v_1[:,38,:],axis=0)
        ice_all = np.expand_dims(ice_1,axis=0)
        #oxy_all_E3SM   = oxy_1
    else:
        if lat_1.shape[0]==363: # 2nd to last year
            fill_array = np.nan*np.ones([1,lat_1.shape[2]])
            fill_array = np.ma.masked_where(np.isnan(fill_array),fill_array)
            lat_1 = np.concatenate((lat_1[:,0,:],fill_array))
            lon_1 = np.concatenate((lon_1[:,0,:],fill_array))
            no3_1 = np.concatenate((no3_1[:,0,:],fill_array))
            u_1 = np.concatenate((u_1[:,38,:],fill_array))
            v_1 = np.concatenate((v_1[:,38,:],fill_array))
            ice_1 = np.concatenate((ice_1,fill_array))
            lat_all = np.concatenate((lat_all,np.expand_dims(lat_1,axis=0)))
            lon_all = np.concatenate((lon_all,np.expand_dims(lon_1,axis=0)))
            no3_all = np.concatenate((no3_all,np.expand_dims(no3_1,axis=0)))
            u_all = np.concatenate((u_all,np.expand_dims(u_1,axis=0)))
            v_all = np.concatenate((v_all,np.expand_dims(v_1,axis=0)))
            ice_all = np.concatenate((ice_all,np.expand_dims(ice_1,axis=0)))
        else:
            lat_all = np.concatenate((lat_all,np.expand_dims(lat_1[:,0,:],axis=0)))
            lon_all = np.concatenate((lon_all,np.expand_dims(lon_1[:,0,:],axis=0)))
            no3_all = np.concatenate((no3_all,np.expand_dims(no3_1[:,0,:],axis=0)))
            ice_all = np.concatenate((ice_all,np.expand_dims(ice_1,axis=0)))
            u_all = np.concatenate((u_all,np.expand_dims(u_1[:,38,:],axis=0)))
            v_all = np.concatenate((v_all,np.expand_dims(v_1[:,38,:],axis=0)))
            
        #oxy_all_E3SM = np.concatenate((oxy_all_E3SM,oxy_1))
        
    #if yy==0: # first time
    #    lat_all = lat_1[:,0,:]
    #    lon_all = lon_1[:,0,:]
    #    no3_all = no3_1[:,0,:]
    #    ice_all = ice_1
    #    #oxy_all_E3SM   = oxy_1
    #else:
    #    lat_all = np.concatenate((lat_all,lat_1[:,0,:]))
    #    no3_all = np.concatenate((no3_all,no3_1[:,0,:]))
    #    ice_all = np.concatenate((ice_all,ice_1))
    #    #oxy_all_E3SM = np.concatenate((oxy_all_E3SM,oxy_1))
        
    del lon_1,lat_1,no3_1,ice_1,u_1,v_1
    print(lon_all.shape)
    
print ('done')


In [ ]:
#-----
# How far has each float travelled (horizontally) in those xx months?
#-----

#-------
# functions
#-------

@njit
def get_closest_grid_point(lon_point, lat_point, lon2, lat2):  
    # in all nodes in mesh, return the index of the closest node to lon_point/lat_point
    # lon2 & lat2 are the locations in the new mesh (to be redistributed to)
    # lon2 & lat2 should be in radians
    # numpy needs to be imported outside the function
    
    #from math import sin, cos, sqrt, atan2, radians
    #import numpy as np
    # approximate radius of earth in km
    R = 6373.0
    
    #lat2 = radians(mesh.y2) # all positions in mesh
    #lon2 = radians(mesh.x2)
    #lat2 = [radians(x) for x in mesh.y2]
    #lon2 = [radians(x) for x in mesh.x2]
    lat1 = radians(lat_point)
    lon1 = radians(lon_point)
    bb1 = cos(lat1)
    
    all_distances = np.zeros(len(lon2))
    for i in range(0,len(lon2)):
        dlon = lon2[i] - lon1
        dlat = lat2[i] - lat1
        a = sin(dlat / 2)**2 + bb1 * cos(lat2[i]) * sin(dlon / 2)**2
        all_distances[i] = 2*R*atan2(sqrt(a), sqrt(1 - a)) # to speed things up, omit constant factors here!
        #all_distances[i] = 2*R*atan2(sqrt(a), sqrt(1 - a)) # correct distance
        #del dlon, dlat, a
    index_closest_node = np.argmin(all_distances)
    distance_closest_node = np.min(all_distances)

    return index_closest_node, distance_closest_node

@njit
def get_closest_grid_point_vector(lon_point, lat_point, lon2, lat2):  
    # PROVIDE LIST OF LOCATIONS TO FUNCTION!
    # in all nodes in mesh, return the index of the closest node to lon_point/lat_point
    # lon2 & lat2 are the locations in the new mesh (to be redistributed to)
    # lon2 & lat2 should be in radians
    # numpy needs to be imported outside the function
    
    #from math import sin, cos, sqrt, atan2, radians
    #import numpy as np
    # approximate radius of earth in km
    R = 6373.0
    
    #lat2 = radians(mesh.y2) # all positions in mesh
    #lon2 = radians(mesh.x2)
    #lat2 = [radians(x) for x in mesh.y2]
    #lon2 = [radians(x) for x in mesh.x2]
    
    index_closest_node    = np.zeros(len(lon_point))
    distance_closest_node = np.zeros(len(lon_point))
    for jj in range(0,len(lon_point)):
        lat1 = radians(lat_point[jj])
        lon1 = radians(lon_point[jj])
        bb1 = cos(lat1)
        
        all_distances = np.zeros(len(lon2))
        for i in range(0,len(lon2)):
            dlon = lon2[i] - lon1
            dlat = lat2[i] - lat1
            a = sin(dlat / 2)**2 + bb1 * cos(lat2[i]) * sin(dlon / 2)**2
            all_distances[i] = 2*R*atan2(sqrt(a), sqrt(1 - a)) # to speed things up, omit constant factors here!
            #all_distances[i] = 2*R*atan2(sqrt(a), sqrt(1 - a)) # correct distance
            #del dlon, dlat, a
        index_closest_node[jj] = np.argmin(all_distances)
        distance_closest_node[jj] = np.min(all_distances)

    return index_closest_node, distance_closest_node

@njit
def get_distance_between_two_points(lon_point, lat_point, lon_point2, lat_point2):
        # all coordinates NOT in radians
        #from math import sin, cos, sqrt, atan2, radians
        #import numpy as np
        # approximate radius of earth in km
        R = 6373.0

        #lat2 = radians(mesh.y2) # all positions in mesh
        #lon2 = radians(mesh.x2)
        #lat2 = [radians(x) for x in mesh.y2]
        #lon2 = [radians(x) for x in mesh.x2]
        lat1 = radians(lat_point)
        lon1 = radians(lon_point)
        bb1 = cos(lat1)
        lat2 = radians(lat_point2)
        lon2 = radians(lon_point2)

        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + bb1 * cos(lat2) * sin(dlon / 2)**2
        distance = 2*R*atan2(sqrt(a), sqrt(1 - a)) # to speed things up, omit constant factors here!

        return distance




In [ ]:
#-----
# load full model output
#-----

path1 = '/global/cfs/cdirs/m4003/maltrud/6year/monthlyEulerianAverages/'
# monthlyAverageIceFraction.year0055.nc

year_list = ['55','56','57','58','59','60']

for yy in tqdm(range(0,len(year_list))):
    print('Load year',year_list[yy]) 
    
    file1 = 'monthlyAverageIceFraction.year00'+year_list[yy]+'.nc'
    ff = xr. open_dataset(path1+file1)
    data=ff['timeMonthly_avg_iceAreaCell'].mean(axis=0)
    data2=ff['timeMonthly_avg_iceAreaCell']#.mean(axis=0)
    data2 = np.mean(data2[5:8+1,:],axis=0) # get JJA avg
    if yy==0:
        sea_ice_full     = data
        sea_ice_full_jja = data2
    else:
        sea_ice_full     = sea_ice_full + data
        sea_ice_full_jja = sea_ice_full_jja + data2
    ff.close()
    del data,data2
    
    file1 = 'monthlyAverageEulerianFields.year00'+year_list[yy]+'.nc'
    ff = xr. open_dataset(path1+file1)
    data1=np.squeeze(ff['timeMonthly_avg_velocityMeridional'].mean(axis=0))
    data2=np.squeeze(ff['timeMonthly_avg_velocityZonal'].mean(axis=0)) 
    data1 = data1[:,38] # approximately 1000m
    data2 = data2[:,38]
    if yy==0:
        vel_full = np.sqrt((data1*data1 + data2*data2))
    else:
        vel_full = vel_full + np.sqrt((data1*data1 + data2*data2))
    ff.close()
    del data1,data2
    
    #print(vel_full.shape)
    
sea_ice_full     = np.divide(sea_ice_full,len(year_list))
sea_ice_full_jja = np.divide(sea_ice_full_jja,len(year_list))
vel_full         = np.divide(vel_full,len(year_list))
print('Annual mean sea ice:',np.min(np.array(sea_ice_full)),np.max(np.array(sea_ice_full)))
print('JJA mean sea ice:',np.min(np.array(sea_ice_full_jja)),np.max(np.array(sea_ice_full_jja)))
print(np.min(np.array(vel_full)),np.max(np.array(vel_full)))
print('done')


In [ ]:
#-----
# load full model output
#-----

path1 = '/global/cfs/cdirs/m4003/maltrud/6year/monthlyEulerianAverages/'
# monthlyAverageIceFraction.year0055.nc

year_list = ['55','56','57','58','59','60']

for yy in tqdm(range(0,len(year_list))):
    print('Load year',year_list[yy]) 
    
    file1 = 'monthlyAverageEulerianFields.year00'+year_list[yy]+'.nc'
    ff = xr. open_dataset(path1+file1)
    data=np.squeeze(ff['timeMonthly_avg_ecosysTracers_NO3']) #.mean(axis=0)
    data = np.mean(data[:,:,0],axis=0)
    if yy==0:
        no3_full     = data
    else:
        no3_full     = no3_full + data
    ff.close()
    del data
    
no3_full     = np.divide(no3_full,len(year_list))
print('Annual mean NO3:',np.min(np.array(no3_full)),np.max(np.array(no3_full)))
print('done')


In [ ]:
#----
# plot Southern Ocean, ANNUAL MEAN
#----
res = 1
levels = np.arange(15, 35+res, res)

# MPAS-O mesh for EC30to60E2r2
meshfile = path_mesh+'ocean.EC30to60E2r2.210210.nc'
mpasmesh = MPASMesh(name = 'EC30to60E2r2', filepath = meshfile)
print(mpasmesh)

# Define MPASOMap for run1
mpasomap_run1 = MPASOMap(data = no3_full, name = 'Sea ice concentration',\
                         units = '', mesh = mpasmesh)
print(mpasomap_run1)

save_plots = True
dpicnt = 350

plt.figure(figsize = [6, 5])
m = mpasomap_run1.plot(region = 'SO_highLats', levels = levels,\
                       cmap = cm.Spectral_r, ptype = 'contourf',\
                       cticks=[0,0.2,0.4,0.6,0.8,1],colorbar=False) # pcolor, contourf

#plt.title('Annual mean, 2012-2017')
if save_plots:
    filename = 'NO3_annual_mean_2012_2017_SO.png'
    print(savepath+filename)
    plt.savefig(savepath+filename,dpi = dpicnt, bbox_inches='tight')
    del filename
plt.show()



In [ ]:
#----
# get average sea ice concentration for different regions
#----

latlim = -60

ind_amundsen = np.where((lon>=210) & (lon<300) & (lat<=latlim))[0]
ind_ross     = np.where((lon>=160) & (lon<210) & (lat<=latlim))[0]
ind_eastAA   = np.where((lon>=0) & (lon<160) & (lat<=latlim))[0]
ind_weddell  = np.where(((lon>=300) | (lon<0)) & (lat<=latlim))[0]
a1 = np.array(np.sum(sea_ice_full[ind_amundsen]*area[ind_amundsen])/np.sum(area[ind_amundsen]))
a2 = np.array(np.sum(sea_ice_full[ind_ross]*area[ind_ross])/np.sum(area[ind_ross]))
a3 = np.array(np.sum(sea_ice_full[ind_eastAA]*area[ind_eastAA])/np.sum(area[ind_eastAA]))
a4 = np.array(np.sum(sea_ice_full[ind_weddell]*area[ind_weddell])/np.sum(area[ind_weddell]))

print('Sea ice concentration:')
print('avg Amundsen/Bellingshausen: '+str(100*a1)+'%')
print('avg Ross Sea: '+str(100*a2)+'%')
print('avg east Antarctic: '+str(100*a3)+'%')
print('avg Weddell Sea: '+str(100*a4)+'%')
print('')

b1 = np.array(np.sum(vel_full[ind_amundsen]*area[ind_amundsen])/np.sum(area[ind_amundsen]))
b2 = np.array(np.sum(vel_full[ind_ross]*area[ind_ross])/np.sum(area[ind_ross]))
b3 = np.array(np.sum(vel_full[ind_eastAA]*area[ind_eastAA])/np.sum(area[ind_eastAA]))
b4 = np.array(np.sum(vel_full[ind_weddell]*area[ind_weddell])/np.sum(area[ind_weddell]))
print('Velocity at 1000m:')
print('avg Amundsen/Bellingshausen: '+str(b1)+' m s-1')
print('avg Ross Sea: '+str(b2)+' m s-1')
print('avg east Antarctic: '+str(b3)+' m s-1')
print('avg Weddell Sea: '+str(b4)+' m s-1')



In [ ]:
#----
# plot Southern Ocean, ANNUAL MEAN
#----
res = 0.1
levels = np.arange(0, 1+res, res) # velocity at 1000m

# MPAS-O mesh for EC30to60E2r2
meshfile = path_mesh+'ocean.EC30to60E2r2.210210.nc'
mpasmesh = MPASMesh(name = 'EC30to60E2r2', filepath = meshfile)
print(mpasmesh)

# Define MPASOMap for run1
mpasomap_run1 = MPASOMap(data = sea_ice_full, name = 'Sea ice concentration',\
                         units = '', mesh = mpasmesh)
print(mpasomap_run1)

save_plots = True
dpicnt = 350

plt.figure(figsize = [6, 5])
m = mpasomap_run1.plot(region = 'SO_highLats', levels = levels,\
                       cmap = cm.Blues_r, ptype = 'contourf',\
                       cticks=[0,0.2,0.4,0.6,0.8,1],colorbar=False) # pcolor, contourf

#plt.title('Annual mean, 2012-2017')
if save_plots:
    filename = 'Sea_ice_annual_mean_2012_2017_SO_blues.png'
    print(savepath+filename)
    plt.savefig(savepath+filename,dpi = dpicnt, bbox_inches='tight')
    ##---
    ## PDF
    ##---
    #filename = 'Velocity_annual_mean_1yr_test_SO_b.pdf'
    #print(savepath+filename)
    #plt.savefig(savepath+filename,dpi = dpicnt, bbox_inches='tight',format='pdf')
    
    del filename
plt.show()



In [ ]:
#----
# plot Southern Ocean, JJA MEAN
#----
res = 0.1
levels = np.arange(0, 1+res, res) # velocity at 1000m

# MPAS-O mesh for EC30to60E2r2
meshfile = path_mesh+'ocean.EC30to60E2r2.210210.nc'
mpasmesh = MPASMesh(name = 'EC30to60E2r2', filepath = meshfile)
print(mpasmesh)

# Define MPASOMap for run1
mpasomap_run1 = MPASOMap(data = sea_ice_full_jja, name = 'Sea ice concentration',\
                         units = '', mesh = mpasmesh)
print(mpasomap_run1)

save_plots = True
dpicnt = 350

plt.figure(figsize = [6, 5])
m = mpasomap_run1.plot(region = 'SO_highLats', levels = levels,\
                       cmap = cm.Blues_r, ptype = 'contourf',\
                       cticks=[0,0.2,0.4,0.6,0.8,1],colorbar=False) # pcolor, contourf

#plt.title('Annual mean, 2012-2017')
if save_plots:
    filename = 'Sea_ice_JJA_mean_2012_2017_SO_blues.png'
    print(savepath+filename)
    plt.savefig(savepath+filename,dpi = dpicnt, bbox_inches='tight')
    ##---
    ## PDF
    ##---
    #filename = 'Velocity_annual_mean_1yr_test_SO_b.pdf'
    #print(savepath+filename)
    #plt.savefig(savepath+filename,dpi = dpicnt, bbox_inches='tight',format='pdf')
    
    del filename
plt.show()


#---
# colorbar separately
#---

lon_reg2 = np.arange(-180,180,1)
lat_reg2 = np.arange(-90,90,1)
lon_reg, lat_reg = np.meshgrid(lon_reg2, lat_reg2)
data_plot = np.zeros_like(lon_reg)
height,width = 18,7
fs = 12

fig = plt.figure(figsize=(height,width))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree(central_longitude=-150))
ax.set_extent([-180, 180, -90, 90])
mm1=ax.contourf(lon_reg, lat_reg, data_plot,\
                   levels=levels,extend='both',cmap=cm.Blues_r,transform=ccrs.PlateCarree())
cbar = plt.colorbar(mm1,ax=ax,orientation='vertical',fraction=0.075, pad=0.02,shrink=0.9,ticks=[0,0.2,0.4,0.6,0.8,1])
cbar.set_label('Sea ice concentration',fontsize=fs-2)
cbar.ax.tick_params(labelsize=fs-3)
fig.gca().set_visible(False)
if save_plots:
    filename = 'COLORBAR_Sea_ice_JJA_mean_2012_2017_SO_blues.png'
    print(savepath+filename)
    plt.savefig(savepath+filename,dpi = dpicnt, bbox_inches='tight')
    del filename
plt.show()

#---
# eps
#---
fig = plt.figure(figsize=(height,width))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree(central_longitude=-150))
ax.set_extent([-180, 180, -90, 90])
mm1=ax.contourf(lon_reg, lat_reg, data_plot,\
                   levels=levels,extend='both',cmap=cm.Blues_r,transform=ccrs.PlateCarree())
cbar = plt.colorbar(mm1,ax=ax,orientation='vertical',fraction=0.075, pad=0.02,shrink=0.9,ticks=[0,0.2,0.4,0.6,0.8,1])
cbar.ax.tick_params(labelsize=fs-3)
cbar.ax.set_yticklabels(['', '', '','','',''])
fig.gca().set_visible(False)
if save_plots:
    filename = 'COLORBAR_Sea_ice_JJA_mean_2012_2017_SO_blues.eps'
    print(savepath+filename)
    plt.savefig(savepath+filename,dpi = dpicnt, bbox_inches='tight',format='eps')
    del filename
plt.show()


In [ ]:
#---
# what do I want to do?
#
# How much does a trajectory differ in length when we don't know its position under ice? 
# 2 steps: 
#    daily data, impact of ice
#    10-daily data, impact of ice
#
# quantify the difference in length based on sectors?
#
#

In [ ]:
#----
# assess the impact of sea-ice cover
#----

# loop over particles: 
# if surface ice cover exceeds ice_threshold, assume that we're losing its exact position
# set these instances to NaN
# then, compute the length of the total trajectory along the original and the modified trajectory

#---
# reduce to the Southern Ocean
#---
indSO = np.where(lat_all[0,0,:]<=-50)[0]
lat_all_SO = lat_all[:,:,indSO]
lon_all_SO = lon_all[:,:,indSO]
ice_all_SO = ice_all[:,:,indSO]
no3_all_SO = no3_all[:,:,indSO]
u_all_SO = u_all[:,:,indSO]
v_all_SO = v_all[:,:,indSO]

# 2nd to last year has a day of NaNs
lat_all_SO[4,-1,:] = -999
lon_all_SO[4,-1,:] = -999
ice_all_SO[4,-1,:] = -999
no3_all_SO[4,-1,:] = -999
u_all_SO[4,-1,:] = -999
v_all_SO[4,-1,:] = -999

#print(lat_all_ice_corr.shape)


In [ ]:
#---
# plot INITIAL POSITIONS on a map
#---
# to get an idea whether the floats are evenly spread
from tqdm import tqdm

list_lat = lat_all_SO[:,0,:].ravel()
list_lon = lon_all_SO[:,0,:].ravel()

fs = 10
projection=ccrs.SouthPolarStereo()

fig = plt.figure(figsize=(10, 9))
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_extent([-280, 80, -80, -50], crs=ccrs.PlateCarree())
ax.add_feature(cartopy.feature.LAND,facecolor=("grey"))
ax.coastlines(resolution='50m')
# Compute a circle in axes coordinates, which we can use as a boundary
# for the map. We can pan/zoom as much as we like - the boundary will be
# permanently circular.
make_round_SO_map = True
if make_round_SO_map:
    theta = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpath.Path(verts * radius + center)
    ax.set_boundary(circle, transform=ax.transAxes)

#ind_amundsen = np.where((lon_start>=210) & (lon_start<300))[0]
#ind_ross     = np.where((lon_start>=160) & (lon_start<210))[0]
#ind_eastAA   = np.where((lon_start>=0) & (lon_start<160))[0]
#ind_weddell  = np.where((lon_start>=300) | (lon_start<0))[0]

color1 = 'firebrick' # Amundsen 
color2 = 'mediumpurple' # Ross
color3 = 'cornflowerblue' # eastAA
color4 = 'black' # Weddell

for ff in tqdm(range(0,list_lat.shape[0])):  
    if (list_lon[ff]>=210) & (list_lon[ff]<300):
        color_plot=color1 # Amundsen
    elif (list_lon[ff]>=160) & (list_lon[ff]<210):
        color_plot=color2 # Ross
    elif (list_lon[ff]>=0) & (list_lon[ff]<160):
        color_plot=color3 # eastAA
    elif (list_lon[ff]>=300) | (list_lon[ff]<0):
        color_plot=color4 # Weddell
    mm=ax.plot(list_lon[ff],list_lat[ff],color=color_plot,\
               marker='o',markersize=3,linestyle=None,transform=ccrs.PlateCarree()) 
    
if make_round_SO_map:
    gl=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=.5,\
                    color='grey', alpha=0.8,linestyle='--')#,xlabels_bottom=True)
    gl.top_labels = True
    gl.bottom_labels = True
    gl.left_labels = True
    gl.right_labels=True
    #gl.xlines = True
    gl.xlocator = mticker.FixedLocator([-180,-120,-60,0,60,120]) 
    gl.ylocator = mticker.FixedLocator([-65,-45])
    #gl.ylocator = LatitudeLocator()
    gl.xformatter = LongitudeFormatter()
    gl.yformatter = LatitudeFormatter()
    gl.rotate_labels = False

save_plots = True
if save_plots:
    dpicnt = 200
    filename = 'Map_initial_position_yearly_floats_wColors.png'
    plt.savefig(savepath+filename,dpi = dpicnt, bbox_inches='tight',format='png')#,transparent=True)
    
plt.show()


In [ ]:
#----
# assess the impact of sea-ice cover
#----

# loop over particles: 
# if surface ice cover exceeds ice_threshold, assume that we're losing its exact position
# set these instances to NaN
# then, compute the length of the total trajectory along the original and the modified trajectory

ice_threshold = 0.5
print('ice threshold: '+str(100*ice_threshold)+'%')

lat_all_ice_corr = np.copy(lat_all_SO)
lon_all_ice_corr = np.copy(lon_all_SO)
no3_all_ice_corr = np.copy(no3_all_SO)
u_all_ice_corr = np.copy(u_all_SO)
v_all_ice_corr = np.copy(v_all_SO)

for yy in range(0,lat_all_ice_corr.shape[0]): # all years
    for dd in range(0,lat_all_ice_corr.shape[1]): # all days
        aux = ice_all_SO[yy,dd,:]
        ind = np.where(aux>=ice_threshold)[0]
        lat_all_ice_corr[yy,dd,ind] = np.nan
        lon_all_ice_corr[yy,dd,ind] = np.nan
        no3_all_ice_corr[yy,dd,ind] = np.nan
        u_all_ice_corr[yy,dd,ind] = np.nan
        v_all_ice_corr[yy,dd,ind] = np.nan
        del aux,ind

#print(np.min(ice_all),np.max(ice_all))

# How many float encounter the "high ice" conditions?
ind = np.sum(np.isnan(lat_all_ice_corr),axis=1)
print('ind.shape',ind.shape)
#ind = np.isnan(lat_all_ice_corr)
print('ind min/max:',np.min(ind),np.max(ind))
days_under_high_ice = np.copy(ind)
num_in_ice = np.where(ind>0)[0]
for yy in range(0,lat_all_ice_corr.shape[0]): # all years
    #if yy==4:
    #    print(np.where(ind[yy,:]>0)[0])
    print(100*np.where(ind[yy,:]>0)[0].shape[0]/lat_all_ice_corr.shape[2],'% of all floats in sea-ice area at some point')
#print(num_in_ice.shape[0])
#print(np.sum(np.isnan(lat_all_ice_corr)))

# What impact do interpolated under-ice trajectories have on mapped fields?
# How exactly is the interpolation dealt with in real life? Are the data there and we just don't know the position, thus is the interpolated 
# line divided into XX positions to which the data are attributed?

#----
# How many days do floats spent in "high-ice" conditions
#----
bins = np.arange(0,365+5,5)

fig = plt.figure(figsize=(6,5))
#plt.hist(x, density=True, bins=30)
plt.hist(ind[ind>0],density=False, bins=60)
plt.show()


In [ ]:
#-----
# avg deviation (shading as std) vs. time spent under sea-ice cover
#-----
# step 1: get distance travelled along a) original and b) modified list of positions
# original: lat_all_SO,lon_all_SO
# modified: lat_all_ice_corr,lon_all_ice_corr

## kick out NaNs
#ind = np.where(~np.isnan(lat_all_ice_corr))
#lat_all_ice_corr_no_NaN = lat_all_ice_corr[ind]
#lon_all_ice_corr_no_NaN = lon_all_ice_corr[ind]
#print(lon_all_ice_corr_no_NaN.shape)


# WHY is there so many NaNs???

#----
# get distance trvelled along each trajectory, ignore "zeros" in output
#----

dist_all = np.nan*np.ones([lat_all_SO.shape[0],lat_all_SO.shape[2]]) 
dist_all_ice = np.nan*np.ones([lat_all_ice_corr.shape[0],lat_all_ice_corr.shape[2]])
print(dist_all.shape,dist_all_ice.shape)
for yy in tqdm(range(0,lat_all_SO.shape[0])): # years
    for nn in range(0,lat_all_SO.shape[2]): # floats

        process_ice = True
        if process_ice:
            #-----
            # modified position data ("high ice" regions set to NaN)
            #-----
            aux1 = lat_all_ice_corr[yy,:,nn]
            aux2 = lon_all_ice_corr[yy,:,nn]
            ind = np.where(~np.isnan(aux1))[0]
            aux1 = aux1[ind]
            aux2 = aux2[ind]
            #print(aux1.shape,aux2.shape)
            # loop over all days and get total length of trajectory
            aux3 = 0
            for tt in range(0,aux1.shape[0]-1):
                lon_point1,lat_point1 = aux2[tt],aux1[tt] 
                lon_point2,lat_point2 = aux2[tt+1],aux1[tt+1]           
                aux5 = [lon_point1,lat_point1,lon_point2,lat_point2]
                # only continue if no NaNs involved:
                if len(np.where(np.isnan(aux5))[0])==0:
                    aux3 = aux3 + get_distance_between_two_points(lon_point1,lat_point1,\
                                                                  lon_point2,lat_point2)
                del lon_point1,lat_point1,lon_point2,lat_point2,aux5

            dist_all_ice[yy,nn] = aux3
            del aux3

        #-----
        # original position data
        #-----
        aux1 = lat_all_SO[yy,:,nn]
        aux2 = lon_all_SO[yy,:,nn]
        # NOT SURE IF THE BELOW IS NEEDED (but lat/lon have some NaNs...)
        ind = np.where(~np.isnan(aux1))[0]
        aux1 = aux1[ind]
        aux2 = aux2[ind]
        # loop over all days and get total length of trajectory
        aux3 = 0
        for tt in range(0,aux1.shape[0]-1):
            lon_point1,lat_point1 = aux2[tt],aux1[tt] 
            lon_point2,lat_point2 = aux2[tt+1],aux1[tt+1] 
            aux5 = [lon_point1,lat_point1,lon_point2,lat_point2]
            # only continue if no NaNs involved:
            if len(np.where(np.isnan(aux5))[0])==0:
                aux3 = aux3 + get_distance_between_two_points(lon_point1,lat_point1,\
                                                              lon_point2,lat_point2)
            del lon_point1,lat_point1,lon_point2,lat_point2,aux5
        dist_all[yy,nn] = aux3
        del aux3

print('done')


In [ ]:

#print (days_under_high_ice)

# 
diff_ratio = (dist_all-dist_all_ice)/dist_all
print(diff_ratio.shape)

ind = np.sum(np.isnan(lat_all_ice_corr),axis=1)
days_under_high_ice = np.copy(ind)
print(days_under_high_ice.shape)

#days_under_high_ice = np.delete(days_under_high_ice,ratio>1000000000)
#ratio = np.delete(ratio,ratio>1000000000)

print('Min/Max ratio:',np.nanmin(diff_ratio),np.nanmax(diff_ratio))
print('Min/Max days_under_high_ice:',np.min(days_under_high_ice),np.max(days_under_high_ice))

ind_ice = np.where(days_under_high_ice>0)[0]

fig = plt.figure(figsize=(6,5))
plt.plot(days_under_high_ice[ind_ice],diff_ratio[ind_ice],'ko')
plt.show()

#print(ratio[~np.isnan(ratio)].shape)
#bins = np.arange(1,2.2+0.02,0.02)
#fig = plt.figure(figsize=(6,5))
#plt.hist(ratio,bins,density=True)
#plt.show()



In [ ]:
print(np.nanmax(diff_ratio[:,:]))

In [ ]:
#---
# plot trajectories of particles with large deviation between original and modified 
#---

color_list = ['yellow','mediumpurple','lightgreen','orangered','mediumturquoise','cornflowerblue',\
             'slateblue','lightsteelblue','red','pink','lightgrey','forestgreen','brown','orange',\
             'darkviolet','lightcoral','firebrick','palegreen','palevioletred','navy',\
             'papayawhip','indigo','cyan','bisque','tan','khaki','maroon','darksalmon','olive']
color_list = 30*color_list
print ('Available colors:',len(color_list))

#ind_high_devition2 = np.where(ratio>1.1)[0] # ratio only contains SO parameters
#ind_sort = np.argsort(-1*ratio[ind_high_devition2]) # multiply by -1 to change the order of the sorting (high lats should be plotted last)
#ind_high_devition = ind_high_devition2[ind_sort]
#del ind_sort

cs = 10
fs_text = 9

ratio_threshold = 0.5

ind_high_devition = np.where(diff_ratio>0.1)[0]
print(ind_high_devition.shape)

# for co-location with model grid cells
ind_lat = np.where(lat<=-48)[0]
lat_rad = [radians(x) for x in lat[ind_lat]]
lon_rad = [radians(x) for x in lon[ind_lat]]

#----
plot_example =False
save_plots = False
display_plots = True

counter = 0
for yy in range(0,diff_ratio.shape[0]):
    #ind_high_devition = np.where((diff_ratio[yy,:]>ratio_threshold) & (diff_ratio[yy,:]<1))[0] # not interested in the floats that are 100% under ice
    ind_high_devition = np.where((days_under_high_ice[yy,:]>230) & (days_under_high_ice[yy,:]<232))[0] 
    print(ind_high_devition.shape[0])
    ind_sort = np.argsort(-1*diff_ratio[yy,:][ind_high_devition]) # multiply by -1 to change the order of the sorting (high lats should be plotted last)
    ind_high_devition = ind_high_devition[ind_sort]
    
    for ff in tqdm(range(0,ind_high_devition.shape[0])):
        #print(ff,diff_ratio[yy,ind_high_devition[ff]])
        print('counter,yy,ff:',counter,yy,ff)
        
        aux1 = lat_all_SO[yy,:,ind_high_devition[ff]] #[:,ind_ice]
        aux2 = lon_all_SO[yy,:,ind_high_devition[ff]]

        aux3 = lat_all_ice_corr[yy,:,ind_high_devition[ff]]
        aux4 = lon_all_ice_corr[yy,:,ind_high_devition[ff]]

        aux5 = no3_all_ice_corr[yy,:,ind_high_devition[ff]]
        aux6 = no3_all_SO[yy,:,ind_high_devition[ff]]

        ind_NaN = np.where(np.isnan(aux3))[0]
        ind_noNaN = np.where(~np.isnan(aux3))[0]

        # check how many model grid cells the float passes under ice       
        unique_cells = np.zeros([aux2[ind_NaN].shape[0]])
        lat_under_ice = aux1[ind_NaN]
        lon_under_ice = aux2[ind_NaN]
        index_closest_node, distance_closest_node = get_closest_grid_point_vector(lon_under_ice, lat_under_ice,\
                                                                                          lon_rad, lat_rad)
       
        if plot_example:
            fig = plt.figure(figsize=(10,5))
            #plt.plot(aux2,aux1,'k',marker='o')
            #plt.plot(aux4,aux3,'b',marker='x')

            plt.plot(aux2[ind_noNaN],aux1[ind_noNaN],'k',marker='s',markersize=0.5,zorder=0) # in open water
            #plt.plot(aux2[ind_NaN],aux1[ind_NaN],color_list[ff],marker='x',label='sea-ice conc. > '+str(ice_threshold)) # under ice

            cs = 14
            #edgecolor='orange',
            plt.scatter(aux2[ind_noNaN],aux1[ind_noNaN],s=cs-5,c=aux6[ind_noNaN],marker='s',linestyle='-',label='open water')
            plt.scatter(aux2[ind_NaN],aux1[ind_NaN],s=cs+5,c=aux6[ind_NaN],marker='x',label='sea-ice conc. > '+str(ice_threshold))

            #plt.plot(aux2[ind_noNaN][0],aux1[ind_noNaN][0],'k',marker='o') # in open water

            cbar=plt.colorbar()
            cbar.set_label('Surface NO3 in mmol m$^{-3}$')

            #plt.plot(aux1,aux2,'rx',markersize=cs)
            #plt.plot([lon_all_SO[(ss*10),ff],lon_all_SO[(ss*10)+9,ff]],\
            #                     [lat_all_SO[ss*10,ff],lat_all_SO[(ss*10)+9,ff]],'darkgrey')
            plt.annotate('full trajectory: '+str(np.round(100*dist_all[yy,ind_high_devition[ff]])/100)+' km',\
                            xy=(0.97,0.92),xycoords='axes fraction',fontsize=fs_text,ha='right',color='k')
            plt.annotate('with gaps due to sea-ice presence: '+str(np.round(100*dist_all_ice[yy,ind_high_devition[ff]])/100)+' km',\
                            xy=(0.97,0.86),xycoords='axes fraction',fontsize=fs_text,ha='right',color='k')
            plt.annotate('ratio: '+str(np.round(100*diff_ratio[yy,ind_high_devition[ff]])/100),\
                            xy=(0.97,0.8),xycoords='axes fraction',fontsize=fs_text,ha='right',color='k')
            plt.annotate('total number of days under high ice cover: '+str(days_under_high_ice[yy,ind_high_devition[ff]]),\
                            xy=(0.97,0.74),xycoords='axes fraction',fontsize=fs_text,ha='right',color='k')
            plt.annotate('number of unique model grid cells under high ice cover: '+str(int(np.unique(index_closest_node).shape[0])),\
                            xy=(0.97,0.68),xycoords='axes fraction',fontsize=fs_text,ha='right',color='k')
            plt.ylabel('Latitude in $^{\circ}$N')
            plt.xlabel('Longitude in $^{\circ}$E')

            #handles, labels = ax.get_legend_handles_labels() 
            #order = [2,4,3,1,0] ##specify order of items in legend
            plt.legend(loc='upper center', bbox_to_anchor=(0.65, 1.1),\
                        ncol=2,fancybox=True, frameon=False,shadow=False,prop={'size': 11})

            if save_plots:
                dpicnt = 200
                if counter<10:
                    filename = 'Trajectory_length_SO_only_ice_threshold_'+str(100*ice_threshold)+'_example_0'+str(counter)+'_with_NO3.png'
                else:
                    filename = 'Trajectory_length_SO_only_ice_threshold_'+str(100*ice_threshold)+'_example_'+str(counter)+'_with_NO3.png'
                plt.savefig(savepath+'example_trajectories/'+filename,dpi = dpicnt, bbox_inches='tight',format='png')#,transparent=True)
            counter = counter+1

            if display_plots:
                plt.show()
            else:
                plt.close(fig)
                
        del index_closest_node, distance_closest_node,lat_under_ice,lon_under_ice


#----
# for each example, print the number of grid cells passed in time under ice

                

In [ ]:
#---
# plot trajectories of particles with large deviation between original and modified 
#---

color_list = ['yellow','mediumpurple','lightgreen','orangered','mediumturquoise','cornflowerblue',\
             'slateblue','lightsteelblue','red','pink','lightgrey','forestgreen','brown','orange',\
             'darkviolet','lightcoral','firebrick','palegreen','palevioletred','navy',\
             'papayawhip','indigo','cyan','bisque','tan','khaki','maroon','darksalmon','olive']
color_list = 30*color_list
print ('Available colors:',len(color_list))

#ind_high_devition2 = np.where(ratio>1.1)[0] # ratio only contains SO parameters
#ind_sort = np.argsort(-1*ratio[ind_high_devition2]) # multiply by -1 to change the order of the sorting (high lats should be plotted last)
#ind_high_devition = ind_high_devition2[ind_sort]
#del ind_sort

cs = 10
fs_text = 9

ratio_threshold = 0.5

ind_high_devition = np.where(diff_ratio>0.1)[0]
print(ind_high_devition.shape)

# for co-location with model grid cells
ind_lat = np.where(lat<=-48)[0]
lat_rad = [radians(x) for x in lat[ind_lat]]
lon_rad = [radians(x) for x in lon[ind_lat]]

#----
plot_example =True
save_plots = True
display_plots = True

counter = 0
for yy in range(2,3):#,diff_ratio.shape[0]):
    #ind_high_devition = np.where((diff_ratio[yy,:]>ratio_threshold) & (diff_ratio[yy,:]<1))[0] # not interested in the floats that are 100% under ice
    ind_high_devition = np.where((days_under_high_ice[yy,:]>230) & (days_under_high_ice[yy,:]<232))[0] 
    print(ind_high_devition.shape[0])
    ind_sort = np.argsort(-1*diff_ratio[yy,:][ind_high_devition]) # multiply by -1 to change the order of the sorting (high lats should be plotted last)
    ind_high_devition = ind_high_devition[ind_sort]
    
    for ff in tqdm(range(0,1)): # ind_high_devition.shape[0]
        #print(ff,diff_ratio[yy,ind_high_devition[ff]])
        print('counter,yy,ff:',counter,yy,ff)
        
        aux1 = lat_all_SO[yy,:,ind_high_devition[ff]] #[:,ind_ice]
        aux2 = lon_all_SO[yy,:,ind_high_devition[ff]]

        aux3 = lat_all_ice_corr[yy,:,ind_high_devition[ff]]
        aux4 = lon_all_ice_corr[yy,:,ind_high_devition[ff]]

        aux5 = no3_all_ice_corr[yy,:,ind_high_devition[ff]]
        aux6 = no3_all_SO[yy,:,ind_high_devition[ff]]

        ind_NaN = np.where(np.isnan(aux3))[0]
        ind_noNaN = np.where(~np.isnan(aux3))[0]

        # check how many model grid cells the float passes under ice       
        unique_cells = np.zeros([aux2[ind_NaN].shape[0]])
        lat_under_ice = aux1[ind_NaN]
        lon_under_ice = aux2[ind_NaN]
        index_closest_node, distance_closest_node = get_closest_grid_point_vector(lon_under_ice, lat_under_ice,\
                                                                                          lon_rad, lat_rad)
       
        if plot_example:
            fig = plt.figure(figsize=(11,4)) # 20,5
            
            plt.plot(aux2[ind_noNaN],aux1[ind_noNaN],'k',marker='s',markersize=0.5,zorder=0) # in open water
            
            cs = 50
            plt.scatter(aux2[ind_noNaN],aux1[ind_noNaN],s=cs-5,c=aux6[ind_noNaN],marker='s',linestyle='-',label='open water')
            plt.scatter(aux2[ind_NaN],aux1[ind_NaN],s=cs+5,c=aux6[ind_NaN],marker='x',label='sea-ice conc. > '+str(ice_threshold))

            cbar=plt.colorbar()
            cbar.set_label('Surface NO3 in mmol m$^{-3}$')

            print_text = False
            if print_text:
                plt.annotate('full trajectory: '+str(np.round(100*dist_all[yy,ind_high_devition[ff]])/100)+' km',\
                                xy=(0.97,0.92),xycoords='axes fraction',fontsize=fs_text,ha='right',color='k')
                plt.annotate('with gaps due to sea-ice presence: '+str(np.round(100*dist_all_ice[yy,ind_high_devition[ff]])/100)+' km',\
                                xy=(0.97,0.86),xycoords='axes fraction',fontsize=fs_text,ha='right',color='k')
                plt.annotate('ratio: '+str(np.round(100*diff_ratio[yy,ind_high_devition[ff]])/100),\
                                xy=(0.97,0.8),xycoords='axes fraction',fontsize=fs_text,ha='right',color='k')
                plt.annotate('total number of days under high ice cover: '+str(days_under_high_ice[yy,ind_high_devition[ff]]),\
                                xy=(0.97,0.74),xycoords='axes fraction',fontsize=fs_text,ha='right',color='k')
                plt.annotate('number of unique model grid cells under high ice cover: '+str(int(np.unique(index_closest_node).shape[0])),\
                                xy=(0.97,0.68),xycoords='axes fraction',fontsize=fs_text,ha='right',color='k')
           # plt.ylabel('Latitude in $^{\circ}$N')
           # plt.xlabel('Longitude in $^{\circ}$E')

            #handles, labels = ax.get_legend_handles_labels() 
            #order = [2,4,3,1,0] ##specify order of items in legend
            plt.legend(loc='upper center', bbox_to_anchor=(0.65, 1.1),\
                        ncol=2,fancybox=True, frameon=False,shadow=False,prop={'size': 11})

            if save_plots:
                dpicnt = 200
                #if counter<10:
                #    filename = 'Trajectory_length_SO_only_ice_threshold_'+str(100*ice_threshold)+'_example_0'+str(counter)+'_with_NO3.png'
                #else:
                #    filename = 'Trajectory_length_SO_only_ice_threshold_'+str(100*ice_threshold)+'_example_'+str(counter)+'_with_NO3.png'
                filename = 'FOR_PAPER_Trajectory_length_SO_only_ice_threshold_'+\
                str(100*ice_threshold)+'_example_with_NO3_2.eps'
                plt.savefig(savepath+'example_trajectories/'+filename,dpi = dpicnt, bbox_inches='tight',format='eps')#,transparent=True)
            counter = counter+1

            if display_plots:
                plt.show()
            else:
                plt.close(fig)
                
        del index_closest_node, distance_closest_node,lat_under_ice,lon_under_ice


#----
# for each example, print the number of grid cells passed in time under ice

                

In [ ]:
print(savepath)

In [ ]:
#----
# get the average difference within each sector
#----

#-----
# for the locations under ice, how many model grid cells are passed?
#  colocate each trajectory with closest model grid cell, count how many unique grid cells exist
# 
# divide into sectors (based on lat/lon on day 1): 
#   1) Ross/Amundsen/Bellinghausen (western border: 160°E, eastern border: 60°W)
#   2) Weddell Sea (western border: 60°W, eastern border: 30°E)
#   3) east Antarctic (western border: 30°E, eastern border: 160°E)
#

# things one can look at (with a longer run at higher resolution):
#    IAV
#    impact of linear interpolation on tracer distributions


#for yy in range(0,1):#diff_ratio.shape[0]):
#    
    ##ind_high_devition = np.where((diff_ratio[yy,:]>ratio_threshold) & (diff_ratio[yy,:]<1))[0] # not interested in the floats that are 100% under ice
    #ind_high_devition = np.where((days_under_high_ice[yy,:]>180) & (days_under_high_ice[yy,:]<240))[0] 
    #print(ind_high_devition.shape[0])
    #ind_sort = np.argsort(-1*diff_ratio[yy,:][ind_high_devition]) # multiply by -1 to change the order of the sorting (high lats should be plotted last)
    #ind_high_devition = ind_high_devition[ind_sort]
    
    

In [ ]:
#---
# get average sea ice cover and velocity in each sector
#---

ice_threshold_list = [0.01]

# avg velocity -> I am not sure this works this way (no area-weighting etc.)
avg_vel_amundsen = np.zeros([len(ice_threshold_list)])
avg_vel_ross     = np.zeros([len(ice_threshold_list)])
avg_vel_weddell  = np.zeros([len(ice_threshold_list)])
avg_vel_eastAA   = np.zeros([len(ice_threshold_list)])
# avg sea ice cover -> I am not sure this works this way (no area-weighting etc.)
avg_ice_amundsen = np.zeros([len(ice_threshold_list)])
avg_ice_ross     = np.zeros([len(ice_threshold_list)])
avg_ice_weddell  = np.zeros([len(ice_threshold_list)])
avg_ice_eastAA   = np.zeros([len(ice_threshold_list)])
for ii in range(0,1):#len(ice_threshold_list)):
    ice_threshold = ice_threshold_list[ii]
    print('')
    print('ice threshold: '+str(100*ice_threshold)+'%')

    lat_all_ice_corr = np.copy(lat_all_SO)
    lon_all_ice_corr = np.copy(lon_all_SO)
    no3_all_ice_corr = np.copy(no3_all_SO)
    ice_all_ice_corr = np.copy(ice_all_SO)
    u_all_ice_corr = np.copy(u_all_SO)
    v_all_ice_corr = np.copy(v_all_SO)

    lat_start = lat_all_SO[:,0,:]
    lon_start = lon_all_SO[:,0,:]
    
    for yy in range(0,lat_all_ice_corr.shape[0]): # all years
        for dd in range(0,lat_all_ice_corr.shape[1]): # all days
            aux = ice_all_SO[yy,dd,:]
            ind = np.where(aux>=ice_threshold)[0]
            lat_all_ice_corr[yy,dd,ind] = np.nan
            lon_all_ice_corr[yy,dd,ind] = np.nan
            no3_all_ice_corr[yy,dd,ind] = np.nan
            ice_all_ice_corr[yy,dd,ind] = np.nan
            u_all_ice_corr[yy,dd,ind] = np.nan
            v_all_ice_corr[yy,dd,ind] = np.nan
            del aux,ind
            
    u_all_ice_corr[u_all_ice_corr<-99]=np.nan
    v_all_ice_corr[v_all_ice_corr<-99]=np.nan
    velocity = np.sqrt(np.multiply(u_all_ice_corr,u_all_ice_corr) + np.multiply(v_all_ice_corr,v_all_ice_corr))
    velocity = np.nanmean(velocity,axis=1)
    
    ice = np.nanmean(ice_all_ice_corr,axis=1)

    # impose lat criterion for veolicites (to avoid ACC)
    ind_amundsen2 = np.where((lon_start>=210) & (lon_start<300) & (lat_start<-60))[0]
    ind_ross2     = np.where((lon_start>=160) & (lon_start<210) & (lat_start<-60))[0]
    ind_eastAA2   = np.where((lon_start>=0) & (lon_start<160) & (lat_start<-60))[0]
    ind_weddell2  = np.where(((lon_start>=300) | (lon_start<0)) & (lat_start<-60))[0]
    # get avg velocity at ~1000m for current "high ice" conditions
    avg_vel_amundsen[ii] = np.nanmean(velocity.ravel()[ind_amundsen2])
    avg_vel_ross[ii]     = np.nanmean(velocity.ravel()[ind_ross2])
    avg_vel_weddell[ii]  = np.nanmean(velocity.ravel()[ind_weddell2])
    avg_vel_eastAA[ii]   = np.nanmean(velocity.ravel()[ind_eastAA2])
    # get avg sea ice cover for current "high ice" conditions
    avg_ice_amundsen[ii] = np.nanmean(ice.ravel()[ind_amundsen2])
    avg_ice_ross[ii]     = np.nanmean(ice.ravel()[ind_ross2])
    avg_ice_weddell[ii]  = np.nanmean(ice.ravel()[ind_weddell2])
    avg_ice_eastAA[ii]   = np.nanmean(ice.ravel()[ind_eastAA2])
    
print('done')
    

In [ ]:
print(ice.shape)
print(velocity.shape)


In [ ]:

print('velocity:')
print(avg_vel_ross) 
print(avg_vel_weddell)
print(avg_vel_amundsen)
print(avg_vel_eastAA)

print('')
print('sea ice cover:')
print(avg_ice_ross) 
print(avg_ice_weddell)
print(avg_ice_amundsen)
print(avg_ice_eastAA)



In [ ]:

# divide into sectors (based on lat/lon on day 1): 
#   1) Ross/Amundsen/Bellinghausen (western border: 160°E, eastern border: 60°W)
#   2) Weddell Sea (western border: 60°W, eastern border: 30°E)
#   3) east Antarctic (western border: 30°E, eastern border: 160°E)

#----
# assess the impact of sea-ice cover
#----

# loop over particles: 
# if surface ice cover exceeds ice_threshold, assume that we're losing its exact position
# set these instances to NaN
# then, compute the length of the total trajectory along the original and the modified trajectory

#ice_threshold_list = [0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.95]
ice_threshold_list = [0.025,0.05,0.075,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.925,0.95,0.975]

ice_threshold_list = [0.5]


# trajectory difference
avg_amundsen = np.zeros([len(ice_threshold_list)])
avg_ross     = np.zeros([len(ice_threshold_list)])
avg_weddell  = np.zeros([len(ice_threshold_list)])
avg_eastAA   = np.zeros([len(ice_threshold_list)])
std_amundsen = np.zeros([len(ice_threshold_list)])
std_ross     = np.zeros([len(ice_threshold_list)])
std_weddell  = np.zeros([len(ice_threshold_list)])
std_eastAA   = np.zeros([len(ice_threshold_list)])
for ii in range(0,1):#len(ice_threshold_list)):
    ice_threshold = ice_threshold_list[ii]
    print('')
    print('ice threshold: '+str(100*ice_threshold)+'%')

    lat_all_ice_corr = np.copy(lat_all_SO)
    lon_all_ice_corr = np.copy(lon_all_SO)
    no3_all_ice_corr = np.copy(no3_all_SO)
    u_all_ice_corr = np.copy(u_all_SO)
    v_all_ice_corr = np.copy(v_all_SO)

    for yy in range(0,lat_all_ice_corr.shape[0]): # all years
        for dd in range(0,lat_all_ice_corr.shape[1]): # all days
            aux = ice_all_SO[yy,dd,:]
            ind = np.where(aux>=ice_threshold)[0]
            lat_all_ice_corr[yy,dd,ind] = np.nan
            lon_all_ice_corr[yy,dd,ind] = np.nan
            no3_all_ice_corr[yy,dd,ind] = np.nan
            u_all_ice_corr[yy,dd,ind] = np.nan
            v_all_ice_corr[yy,dd,ind] = np.nan
            del aux,ind
            
    u_all_ice_corr[u_all_ice_corr<-99]=np.nan
    v_all_ice_corr[v_all_ice_corr<-99]=np.nan
    velocity = np.sqrt(np.multiply(u_all_ice_corr,u_all_ice_corr) + np.multiply(v_all_ice_corr,v_all_ice_corr))
    velocity = np.nanmean(velocity,axis=1)
    
    # How many float encounter the "high ice" conditions?
    ind = np.sum(np.isnan(lat_all_ice_corr),axis=1)
    #print('ind.shape',ind.shape)
    #ind = np.isnan(lat_all_ice_corr)
    #print('ind min/max:',np.min(ind),np.max(ind))
    days_under_high_ice = np.copy(ind)
    num_in_ice = np.where(ind>0)[0]

    #----
    # get distance trvelled along each trajectory, ignore "zeros" in output
    #----

    dist_all = np.nan*np.ones([lat_all_SO.shape[0],lat_all_SO.shape[2]]) 
    dist_all_ice = np.nan*np.ones([lat_all_ice_corr.shape[0],lat_all_ice_corr.shape[2]])
    #print(dist_all.shape,dist_all_ice.shape)
    for yy in range(0,lat_all_SO.shape[0]): # years
        for nn in range(0,lat_all_SO.shape[2]): # floats

            process_ice = True
            if process_ice:
                #-----
                # modified position data ("high ice" regions set to NaN)
                #-----
                aux1 = lat_all_ice_corr[yy,:,nn]
                aux2 = lon_all_ice_corr[yy,:,nn]
                ind = np.where(~np.isnan(aux1))[0]
                aux1 = aux1[ind]
                aux2 = aux2[ind]
                #print(aux1.shape,aux2.shape)
                # loop over all days and get total length of trajectory
                aux3 = 0
                for tt in range(0,aux1.shape[0]-1):
                    lon_point1,lat_point1 = aux2[tt],aux1[tt] 
                    lon_point2,lat_point2 = aux2[tt+1],aux1[tt+1]           
                    aux5 = [lon_point1,lat_point1,lon_point2,lat_point2]
                    # only continue if no NaNs involved:
                    if len(np.where(np.isnan(aux5))[0])==0:
                        aux3 = aux3 + get_distance_between_two_points(lon_point1,lat_point1,\
                                                                      lon_point2,lat_point2)
                    del lon_point1,lat_point1,lon_point2,lat_point2,aux5

                dist_all_ice[yy,nn] = aux3
                del aux3

            #-----
            # original position data
            #-----
            aux1 = lat_all_SO[yy,:,nn]
            aux2 = lon_all_SO[yy,:,nn]
            # NOT SURE IF THE BELOW IS NEEDED (but lat/lon have some NaNs...)
            ind = np.where(~np.isnan(aux1))[0]
            aux1 = aux1[ind]
            aux2 = aux2[ind]
            # loop over all days and get total length of trajectory
            aux3 = 0
            for tt in range(0,aux1.shape[0]-1):
                lon_point1,lat_point1 = aux2[tt],aux1[tt] 
                lon_point2,lat_point2 = aux2[tt+1],aux1[tt+1] 
                aux5 = [lon_point1,lat_point1,lon_point2,lat_point2]
                # only continue if no NaNs involved:
                if len(np.where(np.isnan(aux5))[0])==0:
                    aux3 = aux3 + get_distance_between_two_points(lon_point1,lat_point1,\
                                                                  lon_point2,lat_point2)
                del lon_point1,lat_point1,lon_point2,lat_point2,aux5
            dist_all[yy,nn] = aux3
            del aux3

    #print('done')

    diff_ratio = (dist_all-dist_all_ice)/dist_all
    #print(diff_ratio.shape)

    ind = np.sum(np.isnan(lat_all_ice_corr),axis=1)
    days_under_high_ice = np.copy(ind)
    #print(days_under_high_ice.shape)
    #print('Min/Max ratio:',np.nanmin(diff_ratio),np.nanmax(diff_ratio))
    #print('Min/Max days_under_high_ice:',np.min(days_under_high_ice),np.max(days_under_high_ice))
    ind_ice = np.where(days_under_high_ice>0)[0]


    lat_start = lat_all_SO[:,0,:]
    lon_start = lon_all_SO[:,0,:]
    #print('lon_start',lon_start.shape,lon_start.ravel().shape)

    diff_ratio2 = np.copy(diff_ratio)
    ind_not_nan = np.where(~np.isnan(diff_ratio2.ravel()))[0]
    diff_ratio2 = diff_ratio2.ravel()[ind_not_nan]
    lat_start   = lat_start.ravel()[ind_not_nan]
    lon_start   = lon_start.ravel()[ind_not_nan]
    #print('min/max lon:',np.nanmin(lon_start),np.nanmax(lon_start))

    ind_amundsen = np.where((lon_start>=210) & (lon_start<300))[0]
    ind_ross     = np.where((lon_start>=160) & (lon_start<210))[0]
    #ind_ross    = np.where((lon_start>=160) & (lon_start<300))[0]
    #ind_weddell = np.where((lon_start>=300) | (lon_start<30))[0]
    ind_eastAA   = np.where((lon_start>=0) & (lon_start<160))[0]
    ind_weddell  = np.where((lon_start>=300) | (lon_start<0))[0]
    total_num = ind_amundsen.shape[0]+ind_ross.shape[0]+ind_weddell.shape[0]+ind_eastAA.shape[0]
    print('Sum of all:',total_num)
    print('absolute (Amundsen/Ross/Weddell/eastAA):',ind_amundsen.shape[0],ind_ross.shape[0],\
          ind_weddell.shape[0],ind_eastAA.shape[0])
    print('relative:',ind_amundsen.shape[0]/total_num,ind_ross.shape[0]/total_num,\
          ind_weddell.shape[0]/total_num,ind_eastAA.shape[0]/total_num)


    #avg_ross    = np.mean(dist_all.ravel()[ind_ross])
    #avg_weddell = np.mean(dist_all.ravel()[ind_weddell])
    #avg_eastAA  = np.mean(dist_all.ravel()[ind_eastAA])

    #---
    # TO DO:
    # same for sea ice!
    # get average sea ice cover
    # in the end, the sorting of the region will be a combination
    # of where sea ice cover is high and where velocities are high
    #
    
    avg_amundsen[ii] = np.nanmean(diff_ratio2[ind_amundsen])
    avg_ross[ii]     = np.nanmean(diff_ratio2[ind_ross])
    avg_weddell[ii]  = np.nanmean(diff_ratio2[ind_weddell])
    avg_eastAA[ii]   = np.nanmean(diff_ratio2[ind_eastAA])

    std_amundsen[ii] = np.nanstd(diff_ratio2[ind_amundsen])
    std_ross[ii]     = np.nanstd(diff_ratio2[ind_ross])
    std_weddell[ii]  = np.nanstd(diff_ratio2[ind_weddell])
    std_eastAA[ii]   = np.nanstd(diff_ratio2[ind_eastAA])

    print('')
    print('Amundsen Sea avg: '+str(avg_amundsen[ii])+' +/- '+str(std_amundsen[ii]))
    print('Ross Sea avg: '+str(avg_ross[ii])+' +/- '+str(std_ross[ii]))
    print('east Antarctic avg: '+str(avg_eastAA[ii])+' +/- '+str(std_eastAA[ii]))
    print('Weddell Sea avg: '+str(avg_weddell[ii])+' +/- '+str(std_weddell[ii]))

    # ice threshold 50%
    #Sum of all: 4146
    #absolute (Amundsen/Ross/Weddell/eastAA): 1036 693 955 1462
    #relative: 0.24987940183309212 0.1671490593342981 0.23034249879401833 0.3526290400385914
    #
    #Amundsen Sea avg: 0.13102508154296055 +/- 0.2575042553228124
    #Ross Sea avg: 0.18081375483811757 +/- 0.31005018916652866
    #east Antarctic avg: 0.08282937489108787 +/- 0.2001843133775308
    #Weddell Sea avg: 0.22485873569905585 +/- 0.37251976073398535



In [ ]:

#colovelocity 'mediumpurple'
#color2 = 'darkorange'
#color3 = 'cornflowerblue'
#color4 = 'black'
color1 = 'firebrick' # Amundsen 
color2 = 'mediumpurple' # Ross
color3 = 'cornflowerblue' # eastAA
color4 = 'black' # Weddell

fs = 12
ls = '-'

ice_threshold_list2 = [100*x for x in ice_threshold_list]

fig = plt.figure(figsize=(10,4))
#plt.hlines(0,0,100,color='grey',linewidth=1.0)
#plt.errorbar(ice_threshold_list,avg_amundsen,yerr=std_amundsen)
plt.plot(ice_threshold_list2,100*avg_weddell,'o',color=color4,linestyle=ls,label='Weddell')
plt.plot(ice_threshold_list2,100*avg_ross,'o',color=color2,linestyle=ls,label='Ross')
plt.plot(ice_threshold_list2,100*avg_amundsen,'o',color=color1,linestyle=ls,label='Amundsen')
plt.plot(ice_threshold_list2,100*avg_eastAA,'o',color=color3,linestyle=ls,label='east AA')
plt.ylabel('difference in trajectory length in %\n(known vs. unknown position under ice)',fontsize=fs)
plt.xlabel('ice threshold in %\n(position is unknown whenever ice cover exceeds threshold)',fontsize=fs)
plt.xticks([0,20,40,60,80,100],[0,20,40,60,80,100],fontsize=fs)
plt.yticks(fontsize=fs)
plt.xlim((0,100))
plt.ylim((0,45))
plt.legend(frameon=True)
plt.grid()
save_plots = True
if save_plots:
    dpicnt = 200
    filename = 'Difference_trajectory_length_6year_run_different_regions_v2.png'
    plt.savefig(savepath+filename,dpi = dpicnt, bbox_inches='tight',format='png')#,transparent=True)
plt.show()

plot_eps = True
if plot_eps:
    fig = plt.figure(figsize=(10,4))
    #plt.hlines(0,0,100,color='grey',linewidth=1.0)
    #plt.errorbar(ice_threshold_list,avg_amundsen,yerr=std_amundsen)
    plt.plot(ice_threshold_list2,100*avg_weddell,'o',color=color4,linestyle=ls,label='Weddell')
    plt.plot(ice_threshold_list2,100*avg_ross,'o',color=color2,linestyle=ls,label='Ross')
    plt.plot(ice_threshold_list2,100*avg_amundsen,'o',color=color1,linestyle=ls,label='Amundsen')
    plt.plot(ice_threshold_list2,100*avg_eastAA,'o',color=color3,linestyle=ls,label='east AA')
    #plt.ylabel('difference in trajectory length in %\n(known vs. unknown position under ice)',fontsize=fs)
    #plt.xlabel('ice threshold in %\n(position is unknown whenever ice cover exceeds threshold)',fontsize=fs)
    plt.xticks([0,20,40,60,80,100],[],fontsize=fs)
    plt.yticks([0,5,10,15,20,25,30,35,40,45],[],fontsize=fs)
    plt.yticks(fontsize=fs)
    plt.xlim((0,100))
    plt.ylim((0,45))
    plt.legend(frameon=True)
    plt.grid()
    save_plots = True
    if save_plots:
        dpicnt = 200
        filename = 'Difference_trajectory_length_6year_run_different_regions_v2.eps'
        plt.savefig(savepath3+filename,dpi = dpicnt, bbox_inches='tight',format='eps')#,transparent=True)
    plt.show()



In [ ]:
print(savepath)

In [ ]:

#-----
# for the locations under ice, how many model grid cells are passed?
#  colocate each trajectory with closest model grid cell, count how many unique grid cells exist
# 
# divide into sectors (based on lat/lon on day 1): 
#   1) Ross/Amundsen/Bellinghausen (western border: 160°E, eastern border: 60°W)
#   2) Weddell Sea (western border: 60°W, eastern border: 30°E)
#   3) east Antarctic (western border: 30°E, eastern border: 160°E)
#

# things one can look at (with a longer run at higher resolution):
#    IAV
#    impact of linear interpolation on tracer distributions

print(area.shape)
print(lat.shape,lon.shape)

print(lat_all_ice_corr.shape,lon_all_ice_corr.shape)

# go through each year and float

# reduce model grid to Southern Ocean (much fewer grid cells to check)
ind_lat = np.where(lat<=-48)[0]

lat_rad = [radians(x) for x in lat[ind_lat]]
lon_rad = [radians(x) for x in lon[ind_lat]]
        
unique_cells = np.zeros([lon_all_ice_corr.shape[0],lon_all_ice_corr.shape[2]])
for yy in range(0,lon_all_ice_corr.shape[0]):
    for nn in tqdm(range(0,lon_all_ice_corr.shape[2])):
        aux1 = lat_all_ice_corr[yy,:,nn]
        aux2 = lon_all_ice_corr[yy,:,nn]
        ind_NaN = np.where(np.isnan(aux1))[0]
        #print(ind_NaN.shape)
        
        lat_under_ice = lat_all_SO[yy,ind_NaN,nn]
        lon_under_ice = lon_all_SO[yy,ind_NaN,nn]
        #print(lat_under_ice.shape)
        
        index_closest_node, distance_closest_node = get_closest_grid_point_vector(lon_under_ice, lat_under_ice,\
                                                                                  lon_rad, lat_rad)
        
        unique_cells[yy,nn] = int(np.unique(index_closest_node).shape[0])
        #print(unique_cells[yy,nn])
        
        del index_closest_node, distance_closest_node,lat_under_ice,lon_under_ice,aux1,aux2,ind_NaN
        


In [ ]:
#print(index_closest_node)
print(unique_cells.shape,np.min(unique_cells),np.max(unique_cells))

data_mean = np.mean(unique_cells,axis=0)
data_max  = np.max(unique_cells,axis=0)
print(data_max.shape)

v1,v2=1,15

fig = plt.figure(figsize=(18,3))
plt.plot(np.arange(0,data_mean.shape[0]),data_mean,'b')
plt.plot(np.arange(0,data_max.shape[0]),data_max,'ko')
plt.show()

#fig = plt.figure(figsize=(10,5))
#plt.pcolor(unique_cells,vmin=v1,vmax=v2,cmap=plt.cm.RdYlBu_r)
#plt.colorbar()
#plt.show()



In [ ]:
#---
# plot on a map
#---

fs = 10
projection=ccrs.SouthPolarStereo()

fig = plt.figure(figsize=(10, 9))
ax = fig.add_subplot(1, 1, 1, projection=projection)
#ax.set_extent([-100, 30, -80, -60]) # Weddell Sea
#ax.set_extent([-220, -120, -85, -60]) # Ross Sea
#ax.set_extent([-170, -150, -76, -70])
ax.set_extent([-280, 80, -80, -60], crs=ccrs.PlateCarree())
ax.add_feature(cartopy.feature.LAND,facecolor=("grey"))
ax.coastlines(resolution='50m')
# Compute a circle in axes coordinates, which we can use as a boundary
# for the map. We can pan/zoom as much as we like - the boundary will be
# permanently circular.
make_round_SO_map = True
if make_round_SO_map:
    theta = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpath.Path(verts * radius + center)
    ax.set_boundary(circle, transform=ax.transAxes)

for ff in range(0,ind_high_devition.shape[0]):
    
    aux1 = lat_all_SO[:,ind_high_devition[ff]]
    aux2 = lon_all_SO[:,ind_high_devition[ff]]
    
    aux3 = lat_all_ice_corr[:,ind_high_devition[ff]]
    aux4 = lon_all_ice_corr[:,ind_high_devition[ff]]
    
    ind_NaN = np.where(np.isnan(aux3))[0]
    ind_noNaN = np.where(~np.isnan(aux3))[0]
    
    mm=ax.plot(aux2[ind_noNaN],aux1[ind_noNaN],'k',marker='o',linestyle=None,transform=ccrs.PlateCarree()) # in open water
    mm=ax.plot(aux2[ind_NaN],aux1[ind_NaN],color_list[ff],marker='x',linestyle='none',transform=ccrs.PlateCarree()) # under ice

if make_round_SO_map:
    gl=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=.5,\
                    color='grey', alpha=0.8,linestyle='--')#,xlabels_bottom=True)
    gl.top_labels = True
    gl.bottom_labels = True
    gl.left_labels = True
    gl.right_labels=True
    #gl.xlines = True
    gl.xlocator = mticker.FixedLocator([-180,-120,-60,0,60,120]) 
    gl.ylocator = mticker.FixedLocator([-65,-45])
    #gl.ylocator = LatitudeLocator()
    gl.xformatter = LongitudeFormatter()
    gl.yformatter = LatitudeFormatter()
    gl.rotate_labels = False

save_plots = False

if save_plots:
    dpicnt = 200
    filename = 'Map_trajectory_length_SO_only_ice_threshold_'+str(100*ice_threshold)+'_example_'+str(counter)+'.png'
    plt.savefig(savepath+filename,dpi = dpicnt, bbox_inches='tight',format='png')#,transparent=True)
    
plt.show()

